In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

# nltk.download('punkt')
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

aspects = ['camera', 'battery', 'screen', 'performance', 'speed', 'design', 'sound', 'quality']

input_file = "total_phone_reviews.csv"
output_file = "total_review_sentiments.csv"

reviews_df = pd.read_csv(input_file)

if 'review' not in reviews_df.columns or 'model' not in reviews_df.columns:
    raise ValueError("The input CSV file must contain 'model' and 'review' columns.")

def extract_aspects(review, aspect_keywords):
    tokens = word_tokenize(review.lower())

    found_aspects = [aspect for aspect in aspect_keywords if aspect in tokens]

    return found_aspects

def classify_sentiment_and_phrase(review, aspect):
    sentences = sent_tokenize(review.lower())

    for sentence in sentences:
        if aspect in sentence:
            sentiment_score = sia.polarity_scores(sentence)
            compound_score = sentiment_score['compound']

            if compound_score >= 0.05:
                sentiment = 'Positive'
            elif compound_score <= -0.05:
                sentiment = 'Negative'
            else:
                sentiment = 'Neutral'

            return sentiment, sentence

    return None, None

sentiment_results = []

for index, row in reviews_df.iterrows():
    review = row['review']
    model = row['model']

    if isinstance(review, str):
        aspects_found = extract_aspects(review, aspects)

        review_sentiments = {'model': model, 'review': review}

        for aspect in aspects_found:
            sentiment, full_phrase = classify_sentiment_and_phrase(review, aspect)

            if sentiment:
                review_sentiments[aspect] = sentiment
                review_sentiments[f"{aspect}_full_phrase"] = full_phrase

        sentiment_results.append(review_sentiments)
    else:
        print(f"Skipping invalid review: {review}")

sentiment_df = pd.DataFrame(sentiment_results)

sentiment_df.to_csv(output_file, index=False)

print(f"Sentiment analysis completed. Results saved to '{output_file}'.")
